In [1]:
import tensorflow as tf
import keras
#import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import json
import pandas as pd
import verbose_amalgam_predict as pred
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam #SGD, Adadelta, RMSprop
from keras import layers
from tensorflow.keras import regularizers
#from sklearn import preprocessing

loaded 6823 triples, which use 6823 names
Separated into data_triples: 3984 and test_triples: 463
Model: "factorization_model1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6823)]       0           []                               
                                                                                                  
 repr (Dense)                   (None, 10)           68240       ['input_1[0][0]']                
                                                                                                  
 prob_self (Dense)              (None, 1)            11          ['repr[0][0]']                   
                                                                                                  
 low_rank_out (Dense)           (None, 6823)         75053       ['repr[0][0]'] 

In [5]:
# Open and Read JSON file's data

file = open(r"shallow-frequencies/shallow-frequencies.json", "r", encoding='utf-8')
js = file.read()
data = json.loads(js)
file.close()

In [6]:
# Get all the proofs
proofs = data["triples"]

types = [row["name"] for row in data["types"]]
terms = [row["name"] for row in data["terms"]]

In [7]:
input_list = []
output_list = []

len_types = len(types)
len_terms = len(terms)

for proof in proofs:
    input_def = [0]*len_types
    output_def = [0]*len_terms
    
    typs = set(proof['types'])
    for typ in typs:
        input_def[types.index(typ)] += proof['types'].count(typ)
    input_list.append(input_def)

    trms = set(proof['terms'])
    for trm in trms:
        output_def[terms.index(trm)] += proof['types'].count(trm)
    output_list.append(output_def)

In [8]:
input_list = np.asarray(input_list)
output_list = np.asarray(output_list)

In [9]:
#no_test = 300 no_test/len(proofs)
x_train, x_test, y_train, y_test = train_test_split(input_list, output_list, test_size=0.15)

In [10]:
model = Sequential()

model.add(Dense(200, input_shape = (len_types,), activation = "softplus"))
model.add(Dropout(0.2))

model.add(Dense(len_terms, activation = "softplus"))
model.add(Dropout(0.05))

model.compile(Adam(learning_rate = 0.002), "categorical_crossentropy", metrics = ["KLDivergence"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               142000    
                                                                 
 dropout_12 (Dropout)        (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 3337)              670737    
                                                                 
 dropout_13 (Dropout)        (None, 3337)              0         
                                                                 
Total params: 812,737
Trainable params: 812,737
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_train, y_train, epochs = 50, validation_data=(x_test, y_test))

Epoch 1/50
182/182 [==============================] - 5s 9ms/step - loss: 5.0758 - kullback_leibler_divergence: 1.3587 - val_loss: 3.3608 - val_kullback_leibler_divergence: 1.0802
Epoch 2/50
182/182 [==============================] - 1s 7ms/step - loss: 3.8886 - kullback_leibler_divergence: 1.6977 - val_loss: 2.7910 - val_kullback_leibler_divergence: 1.4108
Epoch 3/50
182/182 [==============================] - 1s 7ms/step - loss: 3.4193 - kullback_leibler_divergence: 1.7561 - val_loss: 2.4103 - val_kullback_leibler_divergence: 1.3932
Epoch 4/50
182/182 [==============================] - 1s 6ms/step - loss: 2.9749 - kullback_leibler_divergence: 1.6733 - val_loss: 2.2072 - val_kullback_leibler_divergence: 1.3704
Epoch 5/50
182/182 [==============================] - 1s 6ms/step - loss: 2.8801 - kullback_leibler_divergence: 1.5836 - val_loss: 2.0761 - val_kullback_leibler_divergence: 1.2953
Epoch 6/50
182/182 [==============================] - 1s 6ms/step - loss: 2.5495 - kullback_leibler_

In [17]:
pred.fit(100, pred.model2)

Epoch 1/100
63/63 [==============================] - 3s 23ms/step - loss: 6.7652 - kullback_leibler_divergence: 6.7147 - val_loss: 6.3686 - val_kullback_leibler_divergence: 6.3041
Epoch 2/100
63/63 [==============================] - 1s 18ms/step - loss: 5.8317 - kullback_leibler_divergence: 5.7168 - val_loss: 4.9335 - val_kullback_leibler_divergence: 4.7950
Epoch 3/100
63/63 [==============================] - 1s 19ms/step - loss: 4.9076 - kullback_leibler_divergence: 4.7819 - val_loss: 4.6228 - val_kullback_leibler_divergence: 4.5032
Epoch 4/100
63/63 [==============================] - 1s 18ms/step - loss: 4.7742 - kullback_leibler_divergence: 4.6550 - val_loss: 4.5742 - val_kullback_leibler_divergence: 4.4564
Epoch 5/100
63/63 [==============================] - 1s 18ms/step - loss: 4.7150 - kullback_leibler_divergence: 4.5976 - val_loss: 4.5547 - val_kullback_leibler_divergence: 4.4373
Epoch 6/100
63/63 [==============================] - 1s 18ms/step - loss: 4.6733 - kullback_leibler_

In [18]:
pred.fit(100, pred.model3)

Epoch 1/100
63/63 [==============================] - 3s 23ms/step - loss: 6.7701 - kullback_leibler_divergence: 6.7194 - val_loss: 6.4535 - val_kullback_leibler_divergence: 6.3972
Epoch 2/100
63/63 [==============================] - 1s 19ms/step - loss: 6.2441 - kullback_leibler_divergence: 6.1504 - val_loss: 5.5931 - val_kullback_leibler_divergence: 5.4648
Epoch 3/100
63/63 [==============================] - 1s 19ms/step - loss: 5.5047 - kullback_leibler_divergence: 5.3709 - val_loss: 4.8560 - val_kullback_leibler_divergence: 4.7235
Epoch 4/100
63/63 [==============================] - 1s 20ms/step - loss: 5.2280 - kullback_leibler_divergence: 5.0970 - val_loss: 4.7384 - val_kullback_leibler_divergence: 4.6088
Epoch 5/100
63/63 [==============================] - 1s 20ms/step - loss: 5.1264 - kullback_leibler_divergence: 4.9951 - val_loss: 4.6677 - val_kullback_leibler_divergence: 4.5374
Epoch 6/100
63/63 [==============================] - 1s 19ms/step - loss: 5.0640 - kullback_leibler_

In [19]:
pred.fit(100, pred.model4)

Epoch 1/100
63/63 [==============================] - 4s 30ms/step - loss: 7.5225 - kullback_leibler_divergence: 6.7784 - val_loss: 7.1677 - val_kullback_leibler_divergence: 6.4642
Epoch 2/100
63/63 [==============================] - 1s 23ms/step - loss: 6.7910 - kullback_leibler_divergence: 6.0743 - val_loss: 5.8802 - val_kullback_leibler_divergence: 5.1689
Epoch 3/100
63/63 [==============================] - 1s 23ms/step - loss: 5.9092 - kullback_leibler_divergence: 5.2418 - val_loss: 5.2557 - val_kullback_leibler_divergence: 4.6266
Epoch 4/100
63/63 [==============================] - 2s 24ms/step - loss: 5.6400 - kullback_leibler_divergence: 5.0515 - val_loss: 5.1175 - val_kullback_leibler_divergence: 4.5640
Epoch 5/100
63/63 [==============================] - 2s 25ms/step - loss: 5.4932 - kullback_leibler_divergence: 4.9659 - val_loss: 5.0109 - val_kullback_leibler_divergence: 4.5148
Epoch 6/100
63/63 [==============================] - 1s 24ms/step - loss: 5.3974 - kullback_leibler_

In [21]:
pred.fit(100, pred.model5)

Epoch 1/100
63/63 [==============================] - 3s 29ms/step - loss: 7.5302 - kullback_leibler_divergence: 6.8354 - val_loss: 7.2489 - val_kullback_leibler_divergence: 6.5943
Epoch 2/100
63/63 [==============================] - 1s 22ms/step - loss: 6.8251 - kullback_leibler_divergence: 6.1526 - val_loss: 5.8552 - val_kullback_leibler_divergence: 5.1901
Epoch 3/100
63/63 [==============================] - 1s 21ms/step - loss: 5.9605 - kullback_leibler_divergence: 5.3331 - val_loss: 5.3661 - val_kullback_leibler_divergence: 4.7707
Epoch 4/100
63/63 [==============================] - 1s 22ms/step - loss: 5.7201 - kullback_leibler_divergence: 5.1614 - val_loss: 5.2137 - val_kullback_leibler_divergence: 4.6876
Epoch 5/100
63/63 [==============================] - 1s 23ms/step - loss: 5.5605 - kullback_leibler_divergence: 5.0583 - val_loss: 5.0968 - val_kullback_leibler_divergence: 4.6267
Epoch 6/100
63/63 [==============================] - 1s 23ms/step - loss: 5.4625 - kullback_leibler_

In [2]:
pred.fit(100, pred.model6)

Epoch 1/100
63/63 [==============================] - 3s 23ms/step - loss: 6.7265 - kullback_leibler_divergence: 6.5171 - val_loss: 5.4677 - val_kullback_leibler_divergence: 5.2409
Epoch 2/100
63/63 [==============================] - 1s 19ms/step - loss: 5.3361 - kullback_leibler_divergence: 5.1496 - val_loss: 4.7278 - val_kullback_leibler_divergence: 4.5626
Epoch 3/100
63/63 [==============================] - 1s 19ms/step - loss: 5.0945 - kullback_leibler_divergence: 4.9302 - val_loss: 4.6575 - val_kullback_leibler_divergence: 4.4962
Epoch 4/100
63/63 [==============================] - 1s 19ms/step - loss: 5.0120 - kullback_leibler_divergence: 4.8489 - val_loss: 4.6152 - val_kullback_leibler_divergence: 4.4528
Epoch 5/100
63/63 [==============================] - 1s 19ms/step - loss: 4.9537 - kullback_leibler_divergence: 4.7898 - val_loss: 4.5983 - val_kullback_leibler_divergence: 4.4334
Epoch 6/100
63/63 [==============================] - 1s 19ms/step - loss: 4.9224 - kullback_leibler_